# Obtaining Your Data - Lab

## Introduction
In this lab you'll practice your munging and transforming skills in order to load in your data to solve a regression problem.

## Objectives
You will be able to:
* Understand the ETL process and the steps it consists of
* Understand the challenges of working with data from multiple sources 

## Task Description

You just got hired by Lego! Your first project is going to be to develop a pricing algorithm to help set a target price for new lego sets that are released to market. To do this, you're first going to need to start mining the company database in order to collect the information you need to develop a model.

Start by investigating the database stored in lego.db and joining the tables into a unified dataset!

> **Hint:** use this sql statement to preview the tables in an unknown database:
```sql
SELECT name FROM sqlite_master
             WHERE type='table'
             ORDER BY name;
```

In [82]:
# Your code here

import pandas as pd
import psycopg2

In [84]:
!createdb lego2

In [85]:
! which pgloader

/usr/local/bin/pgloader


In [86]:
!pgloader ./lego.db postgresql:///lego2

2019-08-13T01:24:53.016000+01:00 LOG pgloader version "3.6.1"
2019-08-13T01:24:53.019000+01:00 LOG Data errors in '/private/tmp/pgloader/'
2019-08-13T01:24:53.157000+01:00 LOG Migrating from #<SQLITE-CONNECTION sqlite:///Users/flatironschool/development/mod4/learnco/dsc-obtaining-your-data-lab-seattle-ds-062419/./lego.db {1005CE4913}>
2019-08-13T01:24:53.157000+01:00 LOG Migrating into #<PGSQL-CONNECTION pgsql://flatironschool@UNIX:5432/lego2 {1005E26143}>
2019-08-13T01:24:53.835000+01:00 LOG report summary reset
             table name     errors       rows      bytes      total time
-----------------------  ---------  ---------  ---------  --------------
                  fetch          0          0                     0.000s
        fetch meta data          0          8                     0.054s
         Create Schemas          0          0                     0.001s
       Create SQL Types          0          0                     0.007s
          Create tables          0         

In [92]:
conn = psycopg2.connect(host='localhost',database='lego2')
c = conn.cursor()

In [130]:
conn.rollback()

In [131]:
response = ['product_pricing', 'product_info', 'product_details','product_reviews']

In [138]:
for i in response:
    c.execute(f"""SELECT * FROM {i};""")
    df = pd.DataFrame(c.fetchall())
    df.columns= [i[0] for i in c.description]
    print(i,'\n', df,'\n')

product_pricing 
        prod_id country         list_price
0        75823      US              29.99
1        75822      US              19.99
2        75821      US              12.99
3        21030      US              99.99
4        21035      US              79.99
5        21039      US              59.99
6        21028      US              59.99
7        21029      US              49.99
8        21034      US              39.99
9        21033      US              39.99
10       21036      US              39.99
11       21019      US              34.99
12       21032      US              29.99
13       17101      US             159.99
14       41597      US              29.99
15       41614      US              19.99
16       41613      US              19.99
17       41609      US               9.99
18       41608      US               9.99
19       41604      US               9.99
20       41605      US               9.99
21       41606      US               9.99
22       41607  

In [144]:
c.execute("""
            select * from product_pricing
            join product_info using(prod_id)
            join product_details using(prod_id)
            join product_reviews using(prod_id);
            """
         )
joinedlego = pd.DataFrame(c.fetchall())
joinedlego.columns = [i[0] for i in c.description]
joinedlego.head()                                 

,prod_id,country,list_price,ages,piece_count,set_name,prod_desc,prod_long_desc,theme_name,num_reviews,play_star_rating,review_difficulty,star_rating,val_star_rating
0,75823,US,29.99,6-12,277,Bird Island Egg Heist,blah,blahblah,Angry Birds™,2.0,4.0,Average,4.5,4.0
1,75822,US,19.99,6-12,168,Piggy Plane Attack,blah,blahblah,Angry Birds™,2.0,4.0,Easy,5.0,4.0
2,75821,US,12.99,6-12,74,Piggy Car Escape,blah,blahblah,Angry Birds™,11.0,4.3,Easy,4.3,4.1
3,21030,US,99.99,12+,1032,United States Capitol Building,blah,blahblah,Architecture,23.0,3.6,Average,4.6,4.3
4,21035,US,79.99,12+,744,Solomon R. Guggenheim Museum®,blah,blahblah,Architecture,14.0,3.2,Challenging,4.6,4.1


In [147]:
joinedlego.to_csv('joinedlego.csv', index=False)

## Summary
Nice work! You're working more and more independently through the workflow and ensuring data integrity!